# Projet 4

In [5]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import requests
import json
import multiprocessing

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [6]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"


def get_prices_from_url(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content,'html.parser')
    str_global_class = "pure-u-1 pure-u-md-1-3" if url == URL_PAGE2 else "pure-u-1 pure-u-md-1-4"
    global_class = soup.find_all(class_=str_global_class)
    prices = []
    storage = []
    databases = []
    names = []
    dict_out = {}

    i=0

    for class_ in global_class:
        sub_class_price = class_.find_all(class_="pricing-table-price")
        sub_class_names = class_.find_all("h2")
        sub_class_storage = class_.find_all(class_="pricing-table-list")[0].find_all("li")
        sub_class_storage = class_.find_all(class_="pricing-table-list")[0].find_all("li")

        prices.append(sub_class_price[0].text.split()[0])
        storage.append(sub_class_storage[3].text.split()[0])
        databases.append(int(sub_class_storage[4].text.split()[0]))
        names.append(sub_class_names[0].text)
        i+=1

    for i in range (0,len(prices)):
        dict_temp={}
        dict_temp['price']=prices[i]
        dict_temp['storage']=storage[i]
        dict_temp['databases']=databases[i]
        dict_out[names[i]]=dict_temp

    return dict_out

In [7]:
get_prices_from_url(URL_PAGE3)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25},
 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}

Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [23]:
def extract_beer_infos(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content,'html.parser')

    infos = {
        'name': soup.find(class_="product-detail-info-title").find("h1").text,
        'note': soup.find(class_="stars")['data-percent'],
        'price': soup.find_all(class_="price")[0].text.split()[0].replace(",","."),
        'volume': int(soup.find(class_="small-6 medium-9 columns js-beer-volume").text.split()[0]),
    }
    return infos

url = "https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33"
extract_beer_infos(url)


{'name': 'Mélusine Bio', 'note': '70', 'price': '38.99', 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [24]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [36]:

def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beers_pages = []
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    beers_json = json.loads(soup.text)

    for beer in beers_json['items']:
        beers_pages.append('https://www.beerwulf.com'+beer['contentReference'])

    # Sequential version (slow):
    beers = []
    for beer in beers_pages:
        beers.append(extract_beer_infos(beer))

    # Parallel version (faster):
    #print(beers_pages)
    #mpool = multiprocessing.Pool(2)
    #beers = mpool.map(extract_beer_infos, beers_pages)

    return beers

In [37]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [38]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.797s

OK
